<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/DeepLearningFiles/fine_tuning_conversational_model_for_mental_health_problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow -q

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Omdena Bhutan Chapter - Leveraging AI to Combat Mental Health Problems/Tasks/Task 2/combined_data.csv")
df.head()

In [ ]:
df.info()

In [ ]:
ndf = df.dropna()

In [ ]:
ndf.info()

In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
# Clean the text data
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z]+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

In [ ]:
# get the cleaned text
ndf["cleaned_text"] = ndf["text"].apply(clean_text)
ndf.head()

In [ ]:
ndf.drop(columns="text", inplace=True)

In [ ]:
ndf.head()

In [ ]:
ndf["mental_state"].value_counts()

In [ ]:
# create the function to covert the normal state as 1 and rest of the normal state as 0
def sentiment(df,column):
  df[column] = df[column].apply(lambda x: 1 if x == "normal" else 0)
  return df

In [ ]:
ndf["sentiment"] = ndf["mental_state"].apply(lambda x: 1 if x == "normal" else 0)
ndf[ndf["sentiment"] == 1].head()

In [ ]:
ndf["mental_state"].nunique()

## Model Development: Multiple output model for the sentiment and emotion classification

In [ ]:
# let's build keras model with the embeddings of the dataset
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

text = ndf["cleaned_text"].values
labels = ndf["mental_state"].values
label2 = ndf["sentiment"].values

labels_encoded, labels_names = pd.factorize(labels)
labels = to_categorical(labels_encoded, num_classes=len(labels_names))

# tokenize the dataset
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

vocal_size = len(tokenizer.word_index) + 1
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")

In [ ]:
padded_sequences.shape

In [ ]:
# Build model architecture for two prediction outputs
from keras.layers import Dense, Dropout, GRU, BatchNormalization, Input, GlobalMaxPooling1D, Concatenate
from keras.models import Model
from keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout

In [ ]:
def multioutput_model(vocab_size, max_length, embedding_dim, num_emotions, num_sentiment):
  """
  model architecture code for the multioutput model

  Args:
  vocab_size: total number of words in vocabulary
  max_length: maximul lenght of sequence
  embedding_dim: dimension of embedding layer
  num_emotions: total number of emotions
  num_sentiment: total number of sentiments

  Returns:
  model: multioutput model

  """

  input_text =  Input(shape=(max_length,))
  embedding_layer = Embedding(vocab_size, embedding_dim, input_length=max_length)(input_text)
  # lstm for the emotions detection and sentiment detection
  lstm1 = LSTM(128, return_sequences=True)(embedding_layer)
  lstm1 = Dropout(0.2)(lstm1)
  lstm2 = LSTM(64)(lstm1)
  lstm2 = Dropout(0.2)(lstm2)
  pooled_features = GlobalMaxPooling1D()(embedding_layer)
  # combine the features
  combined_features = Concatenate()([lstm2, pooled_features])
  # two dense layers
  dense1 = Dense(128, activation='relu')(combined_features)
  dense1 = BatchNormalization()(dense1)
  dense1 = Dropout(0.2)(dense1)
  dense2 = Dense(64, activation='relu')(dense1)
  dense2 = BatchNormalization()(dense2)
  dense2 = Dropout(0.2)(dense2)
  # emotion detection output dense layers
  emotion_dense1 = Dense(64, activation='relu')(dense2)
  emotion_dense1 = BatchNormalization()(emotion_dense1)
  emotion_dense1 = Dropout(0.2)(emotion_dense1)
  emotion_output = Dense(num_emotions, activation='softmax', name='emotion_output')(emotion_dense1)
  # sentiment detection output layers
  senti_dense1 = Dense(64, activation='relu')(dense2)
  senti_dense1 = BatchNormalization()(senti_dense1)
  senti_dense1 = Dropout(0.2)(senti_dense1)
  senti_output = Dense(num_sentiment, activation='sigmoid', name='senti_output')(senti_dense1)

  model = Model(inputs=input_text, outputs=[emotion_output, senti_output])
  return model

In [ ]:
# import keras metrics
from tensorflow.keras.metrics import SparseCategoricalAccuracy, SparseCategoricalCrossentropy, BinaryAccuracy, BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import keras

In [ ]:
# compile the model
def compile_model(model, learning_rate=0.001):
    """
    Compile the multi-output model with appropriate losses and metrics.

    Args:
        model: Keras model to compile
        learning_rate: Learning rate for optimizer

    Returns:
        Compiled model
    """

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss={
            'emotion_output': SparseCategoricalCrossentropy(),
            'senti_output': BinaryCrossentropy()
        },
        loss_weights={
            'emotion_output': 1.0,
            'senti_output': 1.0
        },
        metrics={
            'emotion_output': [SparseCategoricalAccuracy(name='emotion_accuracy')],
            'senti_output': [BinaryAccuracy(name='sentiment_accuracy')]
        }
    )

    return model

In [ ]:
# compile the model
model = multioutput_model(vocal_size, max_length, 100, 9, 2)
model = compile_model(model)

In [ ]:
model.summary()

In [ ]:
# train the model
label2_one_hot = to_categorical(label2, num_classes=2)

history = model.fit(
        padded_sequences,
        {
            'emotion_output': labels,
            'senti_output': label2_one_hot
        },
        epochs=5,
        batch_size=32,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5)
        ]
    )